In [1]:
import sys
sys.path.append("..")
import nltk 
nltk.download("stopwords")
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\abdo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
from ast import literal_eval
import pandas as pd
import numpy as np

In [3]:
def read_data(filenmae):
    data=pd.read_csv(filenmae,sep='\t')
    data['tags']=data['tags'].apply(literal_eval)
    return data

In [4]:
train=read_data('data/train.tsv')
validation=read_data('data/validation.tsv')
test=pd.read_csv('data/test.tsv',sep='\t')

In [5]:
from nltk.tokenize import word_tokenize
from string import punctuation

stop_words=set(stopwords.words('english'))
table = str.maketrans('','',punctuation)

In [6]:
def textclean(text):
    tokens=word_tokenize(text)
    tokens=[word for word in tokens if word.isalpha()]
    tokens=[w.translate(table) for w in tokens]
    tokens=[word for word in tokens if not word in stop_words]
    token = [word for word in tokens if len(word)>1 ]
    return tokens

In [9]:
contents_train=[]
contents_validation=[]
for index,row in train.iterrows():
    text=row['title'].lower()
    contents_train.append(textclean(text))
    
for index,row in validation.iterrows():
    text=row['title'].lower()
    contents_validation.append(textclean(text))

In [10]:
import gensim
from gensim.models import Word2Vec
n_dim=200

w2v_model_train =Word2Vec(contents_train,min_count=5,size=n_dim)
w2v_model_validation =Word2Vec(contents_validation,min_count=5,size=n_dim)

D:\Anaconda3\lib\site-packages\smart_open\ssh.py:34: UserWarning: paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress
  warnings.warn('paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress')
D:\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [ ]:
#from sklearn.feature_extraction.text import TfidfVectorizer


#vectorizer  = TfidfVectorizer(min_df=10,analyzer=lambda x: x)

#matrix=vectorizer.fit_transform(contents)
#tfidf = dict(zip(vectorizer.get_feature_names(),vectorizer.idf_))


In [24]:
def create_word_vector(l,size,flag):
    vector = np.zeros(size).reshape((1,size))
    count = 0.
    for word in l:
        try:
            if flag == 'train':
                vector += w2v_model_train[word].reshape((1, size)) 
                count+=1
            else:
                vector += w2v_model_validation[word].reshape((1, size)) 
                count+=1
        except KeyError:
            continue
            
    if count!=0:
        vector /= count
    return vector

X_train = []
y_train = []
X_validation = []
y_validation = []
for i in range(len(train)):
    converted_review = create_word_vector(contents_train[i],n_dim,'train')
    #print(converted_review)
    X_train.append(np.array(converted_review))
    y_train.append(train['tags'][i]) 
for i in range(len(validation)):
    converted_review = create_word_vector(contents_validation[i],n_dim,'validation')
    #print(converted_review)
    X_validation.append(np.array(converted_review))
    y_validation.append(validation['tags'][i]) 

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  import sys
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # Remove the CWD from sys.path while we load stuff.


In [25]:
from sklearn.preprocessing import scale

X_train = np.concatenate(X_train)
X_train = scale(X_train)
y_train = np.array(y_train)


X_validation = np.concatenate(X_validation)
X_validation = scale(X_validation)
y_validation = np.array(y_validation)


In [26]:
X_train.shape

(100000, 200)

In [27]:
from collections import Counter
tags_counts=Counter([item for taglist in y_train for item in taglist])

In [28]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer(classes=sorted(tags_counts.keys()))
y_train = mlb.fit_transform(y_train)
y_validation = mlb.fit_transform(y_validation)

In [30]:
y_train[1]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [32]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression, RidgeClassifier
clf = OneVsRestClassifier(SVC())
clf = clf.fit(X_train,y_train)
............++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

In [ ]:
y_val_predicted = clf.predict(X_validation)
y_val_predicted = clf.decision_function(X_validation)

y_val_pred_inversed = mlb.inverse_transform(y_val_predicted)
y_val_inversed = mlb.inverse_transform(y_validation)
for i in range(3):
    print('Title:\t{}\nTrue labels:\t{}\nPredicted labels:\t{}\n\n'.format(
        validation[i],
        ','.join(y_val_inversed[i]),
        ','.join(y_val_pred_inversed[i])
    ))

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score 
from sklearn.metrics import average_precision_score
from sklearn.metrics import recall_score
def print_evaluation_scores(y_val, predicted):
    
    ######################################
    ######### YOUR CODE HERE #############
    ######################################
    print ("Accracy={}".format(accuracy_score(y_val, predicted)), 
        "\n\nF1_macro={}".format(f1_score(y_val, predicted, average='macro')),
        "\n\nF1_micro={}".format(f1_score(y_val, predicted, average='micro')),
        "\n\nF1_wted={}".format(f1_score(y_val, predicted, average='weighted')),
        "\n\nPrecsion_macro={}".format(average_precision_score(y_val, predicted, average='macro')),
        "\n\nPrecsion_micro={}".format(average_precision_score(y_val, predicted, average='micro')),
        "\n\nPrecsion_wted={}".format(average_precision_score(y_val, predicted, average='weighted')))

In [ ]:
print_evaluation_scores(y_validation, y_val_predicted)
